In [140]:
"""
Joao Assalim
Github: JoaoAssalim
https://www.kaggle.com/competitions/titanic/data?select=train.csv
"""

'\nJoao Assalim\nGithub: JoaoAssalim\nhttps://www.kaggle.com/competitions/titanic/data?select=train.csv\n'

In [171]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import r2_score, accuracy_score

In [142]:
train = pd.read_csv("./train.csv")
train = train.drop(["PassengerId", "Name"], axis=1)

In [143]:
train

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,0,3,male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,211536,13.0000,NaN,S
887,1,1,female,19.0,0,0,112053,30.0000,B42,S
888,0,3,female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,1,1,male,26.0,0,0,111369,30.0000,C148,C


In [144]:
train["Age"].median()

28.0

In [145]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Cabin"] = train["Cabin"].fillna(train["Cabin"].value_counts().index[0])
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].value_counts().index[0])

In [146]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Cabin       0
Embarked    0
dtype: int64

In [147]:
train = pd.concat([train.drop("Embarked", axis=1), pd.get_dummies(train["Embarked"])], axis=1)

In [148]:
train = pd.concat([train.drop("Sex", axis=1), pd.get_dummies(train["Sex"])], axis=1)

In [149]:
train = pd.concat([train.drop("Pclass", axis=1), pd.get_dummies(train["Pclass"], prefix="class")], axis=1)

In [150]:
lb = LabelEncoder()

train["Ticket"] = lb.fit_transform(train["Ticket"])
train["Cabin"] = lb.fit_transform(train["Cabin"])


In [151]:
X = train.drop("Survived", axis=1)
y = train["Survived"]

In [152]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [153]:
pipe = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    RandomForestClassifier()
)

In [154]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [155]:
pipe.score(X_valid, y_valid)

0.8044692737430168

In [163]:
test = pd.read_csv('./test.csv')
results = pd.read_csv("./gender_submission.csv")

In [164]:
test = test.drop(["PassengerId", "Name"], axis=1)

In [165]:
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Cabin"] = test["Cabin"].fillna(test["Cabin"].value_counts().index[0])
test["Embarked"] = test["Embarked"].fillna(test["Embarked"].value_counts().index[0])

In [166]:
test = pd.concat([test.drop("Embarked", axis=1), pd.get_dummies(test["Embarked"])], axis=1)
test = pd.concat([test.drop("Sex", axis=1), pd.get_dummies(test["Sex"])], axis=1)
test = pd.concat([test.drop("Pclass", axis=1), pd.get_dummies(test["Pclass"], prefix="class")], axis=1)

test["Ticket"] = lb.fit_transform(test["Ticket"])
test["Cabin"] = lb.fit_transform(test["Cabin"])

In [167]:
pipe.score(test, results["Survived"])

0.9066985645933014

In [168]:
y_pred = pipe.predict(test)

In [170]:
predictions = pd.DataFrame({"PassengerId": results["PassengerId"], "Survived": y_pred})
predictions.to_csv('output.csv', index=False)

In [172]:
ac = accuracy_score(y_pred, results["Survived"])
r2 = r2_score(y_pred, results["Survived"])

ac, r2

(0.9066985645933014, 0.6077950198484301)